In [1]:
import rasterio
import netCDF4 as nc
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [11]:
class NCTOTIFF:
    def __init__(self, file_path):
        self._filepath = file_path
        self._f = nc.Dataset(self._filepath)
        self._lat = self._f.variables['lat_rho'][:-1,:-1] 
        self._lon = self._f.variables['lon_rho'][:-1,:-1]
        self._latmin = np.max(self._lat)
        self._lonmin = np.min(self._lon)
        self._U = self._f.variables['u'][0,-1,:-1,:]
        self._U = self._U[::-1,:]
        self._V = self._f.variables['v'][0,-1,:,:-1]
        self._V = self._V[::-1,:]
        self._nx, self._ny = self._lat.shape[1], self._lon.shape[0]
        self._lonstep = (self._lon[0,-1] - self._lon[0,0]) / self._nx
        self._latstep = (self._lat[-1,0] - self._lat[0,0]) / self._ny

    def SHraw(self, dataNum=10, scale=10):
        """
            Show Raw data

            Args:
                - dataNum(`int`) : split u,v data using number [default = 10]
                - scalse(`int`) : set plot scale using number [default = 10]
        """
        plt.figure(figsize=(10, 8))
        ax = plt.axes(projection=ccrs.PlateCarree())
        u = self._U[::-1,:]
        v = self._V[::-1,:]

        ax.quiver(self._lon[::dataNum,::dataNum], self._lat[::dataNum,::dataNum],u[::dataNum,::dataNum],v[::dataNum,::dataNum],scale=scale)
        ax.coastlines()
        
    def SVtiff(self, saveName):
        """
            Save tiff file

            Args:
                - saveName(`string`) : save name
        """
        direction = (np.arctan2(self._V, -self._U) * (180 / np.pi)+90)%360
        direction.fill_value=-9999
        speed = np.sqrt(self._U**2+self._V**2)
        sname = saveName+'.tif'

        with rasterio.open(
            sname,
            'w',
            driver='GTiff',
            height=self._U.shape[0],
            width=self._U.shape[1],
            count=2,
            dtype=speed.dtype,
            crs='EPSG:4326',
            transform=rasterio.transform.from_origin(self._lonmin,self._latmin,self._lonstep,self._latstep),
        ) as dst:
            dst.write(speed,1)
            dst.write(direction,2)

In [9]:
west = NCTOTIFF(file_path ='/Users/choiyoonjin/Dev/geo-modelweb/Data/MODEL/nc/his_west_20220703_00091.nc')
# south = NCTOTIFF('/Users/choiyoonjin/Dev/geo-modelweb/Data/nc/his_south_20220523_00056.nc')

In [10]:
west._V.shape

(308, 170)

In [8]:
west._V.shape

(308, 170)

In [9]:
west._nx

170

In [6]:
west._ny

308

In [4]:
west._lonstep

0.011045751633983096

In [69]:
west.SVtiff('west')
south.SVtiff('south')

/Users/choiyoonjin/opt/anaconda3/envs/geomweb/lib/python3.8/site-packages/numpy/ma/core.py:6900: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
